In [1]:
import urllib2
import requests
import os
import re
import pandas as pd
from bs4 import BeautifulSoup


# use the cookie raw string to build a cookie set
def setCookies(raw):
    cookies = dict()
    for i in cookieRaw.split('; '):
        cookies.update({i.split('=')[0]:i.split('=')[1]})
    return cookies

# find url of undownloaded files (mp4,txt,srt,pdf) from the lesson page 
def getLessonFileUrls(lessonPageUrl): 
    fileList=[]
    
    r = requests.get(lessonPageUrl,cookies=cookies) # it's necessary to set cookies in this request, because cookies can tell coursera server the validity of this request.
    html = BeautifulSoup((r.text)) # transform the source code of the lesson page into BeautifulSoup format

    with open('course\\'+coursename+'\\course.txt','w+') as f:
        for weekNum,weekTitle in enumerate([week.find('h3').text[2:] for week in html.findAll('div',{"class" : ['course-item-list-header','expanded']})]):
            f.write(('Week'+str(weekNum+1)+' '+weekTitle+'\n').encode('utf-8'))
    for weekNum,lessonesInOneWeek in enumerate(html.findAll('ul',{"class" : 'course-item-list-section-list'})): # iter weeks
        for lessonNum,lesson in enumerate(lessonesInOneWeek):
            idx = 'Week'+str(weekNum+1)+'-'+str(lessonNum+1)
            lessonName = idx+' '+lesson.find('a',{"class" : 'lecture-link'}).text # set the lesson name
            lessonName = re.sub(r"[\/\\\:\*\?\"\<\>\|\n]", "", lessonName.replace(':','\'')) # wipe the invalid character off
            for downloadLinkArea in lesson.findAll(rel="tooltip"): 
                link = downloadLinkArea['href'] # get the download url
                try:
                    extension = re.search(r'(txt|srt|mp4|pdf)',link).group(1) # extract the filetype from link 
                    fileList.append((link,lessonName + "." + extension)) # pair fileurl and filename off
                except:
                    pass

    return fileList

# start downloading
def download(lessons):
    for fileurl,filename in getLessonFileUrls(lessonPageUrl):
        filePath = 'course\\' + coursename + '\\' + filename
        if not os.path.exists(filePath):
            print('downloading: ' + filename)
            # put the online filestream into the local file 
            with open(filePath,'wb+') as f:
                try:
                    if '.mp4' in fileurl:
                        fileurl = requests.get(fileurl,cookies=cookies).url # get redirected url if the url is for a mp4 file
                    r = requests.get(fileurl)
                    f.write(r.content)
                except (urllib2.HTTPError, urllib2.URLError) as e:
                    print('\nProblem occurs')

# Here's your cookie info of your coursera account
cookieRaw = 'csrf_token=W9vS4JTsx1yrwrPinDie; __204u=1537218674-1432445510539; __204r=; _ga=GA1.2.558542752.1432445521; ab-experiments-session=specializations_subhead_field; data-readme-in-class-qqs-announcement-datascitoolbox-013=1; data-readme-in-class-qqs-announcement-exdata-013=1; ab-experiments-user=in_class_qqs%2Cin_class_qqs_button%2Csigtrack_show_cert_name; CSRF3-Token=1436159564.TFlEnUhXkPVywP2E; CAUTH=X3VqBGfwEf2ywYLoDsO8BcjayufF_67C8g_M4AJP1n1ADT17PIiAdA318uBu0o4ntZ8J9zkO34M_i9mCL8i0KA.O077cpVHPuZgypchzKiTZw.2YONtYzgIdJ3qDRmNCqY80meWxyKgf1EOu7hlvh87TbBqU3by5ZPhwlfHuK0m9sHdJJrzVA2-jDpEETmJ0u4AyRfsGPXCvPhSKu4MdP49BeEsy9qtotKzTrviy7ll8EByAF1YLJup1oncDXSI__wCBycz9KTnXxLfgn9WO2bX50; maestro_login_flag=1; serve_netease_974820=0; __utma=158142248.558542752.1432445521.1433488262.1435295566.14; __utmc=158142248; __utmz=158142248.1432705593.7.2.utmcsr=class.coursera.org|utmccn=(referral)|utmcmd=referral|utmcct=/statinference-013/lecture'
cookies = setCookies(cookieRaw)

# coursenames = [,'datascitoolbox-013','devdataprod-013','fmri-003','ml-007','neuralnets-2012-001','predmachlearn-013','regmods-013','devdataprod-013','getdata-013','repdata-013','rprog-013','composition-003'] # set it
coursenames = ['compneuro-003']
for coursename in coursenames:
    lessonPageUrl = 'https://class.coursera.org/'+coursename+'/lecture/'

    # make directory if it doesn't exist
    if not os.path.exists('course'):
        os.makedirs('course')
    if not os.path.exists('course\\'+coursename):
        os.makedirs('course\\'+coursename)

    download(lessonPageUrl)       
print('All done!')


All done!


In [ ]:
import os
for coursename in coursenames:
    for filename in os.listdir('course\\'+coursename):
        if filename.startswith("Week"):
            week = int(filename.split(' ')[0].split('Week')[1])
            os.rename('course\\'+coursename+'\\'+filename,'course\\'+coursename+'\\'+'Week'+str(week+1)+' '+' '.join(filename.split(' ')[1:]))


In [3]:
import urllib2
import requests
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

coursename = 'compneuro-003' # set it
lessonPageUrl = 'https://class.coursera.org/'+coursename+'/lecture/'

# use the cookie raw string to build a cookie set
def setCookies(raw):
    cookies = dict()
    for i in cookieRaw.split('; '):
        cookies.update({i.split('=')[0]:i.split('=')[1]})
    return cookies

# find url of undownloaded files (mp4,txt,srt,pdf) from the lesson page 
def getLessonFileUrls(lessonPageUrl): 
    r = requests.get(lessonPageUrl,cookies=cookies) # it's necessary to set cookies in this request, because cookies can tell coursera server the validity of this request.
    html = BeautifulSoup((r.text)) # transform the source code of the lesson page into BeautifulSoup format
    print(r.text)
    urls= [i['href'] for i in html.findAll(rel="tooltip") if  re.match(r'.*?(txt|srt|mp4|pdf).*?', i['href'])] # find urls of all files
    if os.path.exists('course\\' + coursename + '\\done.txt'):
        with open('course\\' + coursename + '\\done.txt') as f:
            downloaded = f.read().split('\n')
            urls = list(set(urls) - set(downloaded))
    
    lessonFileUrls = zip(*(iter(urls),) * 4) # groups of 4 links 
    return lessonFileUrls

# get the file infomation (name and url) that need to be downloaded
def getDownloadInfo(lessonUrls):
    r = requests.get(filter(lambda url:'mp4' in url, lessonUrls)[0],cookies=cookies) # get the url of mp4 file of this lesson
    
    lessonName = re.search(r'filename=\"(.*?)\"', urllib2.unquote( urllib2.unquote(r.url) )).group(1) # extract filename from the mp4's url
    lessonName = lessonName.replace('.mp4','') # wipe the extension off
    lessonName = re.sub(r"[\/\\\:\*\?\"\<\>\|]", "", lessonName.replace(':','\'')) # wipe the invalid character off
    
    # pair file name and url off
    fileList = [(lessonName+'.mp4', r.url)]
    for extension in ['srt','txt','pdf']:
        try:
            fileList.append((lessonName+'.'+extension, filter(lambda url: extension in url, lesson)[0]))
        except:
            pass

    return fileList

# start downloading
def download(lessons):
    for lesson in lessons:
        fileList = getDownloadInfo(lesson)
        for filename,fileurl in fileList:
            filePath = 'course\\' + coursename + '\\' + filename
            if not os.path.exists(filePath):
                print('downloading: ' + filename)
                # put the online filestream into the local file 
                with open(filePath,'wb+') as f:
                    try:
                        r = requests.get(fileurl)
                        f.write(r.content)
                    except (urllib2.HTTPError, urllib2.URLError) as e:
                        print('\nProblem occurs')
                        
        with open('course\\' + coursename + '\\done.txt','w+') as f:
            for url in lesson:
                f.write(url+'\n')
                        
# Here's your cookie info of your coursera account
cookieRaw = 'network-priority-support=%7B%22error%22%3A%22user_not_in_network%22%7D; csrf_token=inlsxNKfT77hHyf2NCvH; __204r=; __400u=; __204u=6170509493-1428851675972; ab-experiments-user=new_records_page_eocs_banner%2Cin_class_qqs; data-readme-in-class-qqs-announcement-composition-003=1; ab-experiments-session=sigtrack_demonstrated_uses2; CSRF3-Token=1433236950.RMpLToCFJzVFhlE7; __400v=9ef3c754-3911-485d-b5ce-710815b3d143; CAUTH=TkA-PMD9Rr27HLumHSHUm8oSwnzscenf4AyacRCGz4NjNBiRaJMAsMcVI0xdPCTMor3P6KoR4DrGs3lIYRTRnw.8KRHvbv2E46oeqRDupH8Lg.d1xP-bqjXNzoBF-9RSAnhGlybnlbQl0LpEhYntWIlBCiuzx1STqgiXhtDboPZfVTC4SiNHQ7dUL8omFatp3KqQuoKT-qpyOm3XN5Ig9SJDG0WHL0RAzGtlQjTeLTWMjA8czoFH_aAte5Rl9IaMYqo3co9-21CaA3cJS1V0ia_HQ; maestro_login_flag=1; __400vt=1432373040238; serve_netease_974064=1; __utma=158142248.282732513.1428851676.1430464377.1432372965.8; __utmb=158142248.9.10.1432372965; __utmc=158142248; __utmz=158142248.1428997549.2.2.utmcsr=mail.google.com|utmccn=(referral)|utmcmd=referral|utmcct=/_/scs/mail-static/_/js/k=gmail.main.zh_CN.aop5Jfw4utw.O/m=m_i,t,it/am=PiN64_7v_UGMM2SXPlLZv_9-d0mxM8_Lv3sTQLIzAP5v9v8Afg_spy9k/rt=h/d=1/t=zcms/rs=AHGWq9BWNeBcHISg6ukZsAI43YKUUIeuog'
cookies = setCookies(cookieRaw)

lessons = getLessonFileUrls(lessonPageUrl)

# make directory if it doesn't exist
if not os.path.exists('course'):
    os.makedirs('course')
if not os.path.exists('course\\'+coursename):
    os.makedirs('course\\'+coursename)

download(lessons)            
print('All done!')

<!DOCTYPE html><html xmlns:fb="http://ogp.me/ns/fb#" itemtype="http://schema.org"><head><meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=IE7"><meta name="fragment" content="!"><meta name="robots" content="NOODP"><meta charset="utf-8"><meta property="og:site_name" content="Coursera"><meta property="fb:admins" content="727836538,4807654"><meta property="fb:app_id" content="823425307723964"><meta name="twitter:site" content="Coursera"><meta name="twitter:app:name:iphone" content="Coursera"><meta name="twitter:app:name:ipad" content="Coursera"><meta name="twitter:app:name:googleplay" content="Coursera"><meta name="twitter:app:id:iphone" content="id736535961"><meta name="twitter:app:id:ipad" content="id736535961"><meta name="twitter:app:id:googleplay" content="org.coursera.android"><meta name="viewport" content="width=device-width, initial-scale=1"><link rel="apple-touch-icon" sizes="57x57" href="https://d3njjcbhbojbot.cloudfront.net/web/images/favicons/apple-touch-icon-57x57.png"